# Market Data Feed Handler - OrderBook Processing Assignment

## Overview

You're building a component responsible for processing incoming market data messages and maintaining an up-to-date orderbook. The event stream begins with an initial snapshot of the market, followed by incremental updates represented by data blocks.

## Data Structures

### DataBlock
Each DataBlock contains an event type and relevant fields depending on that event.

### Event Types

The four supported event types are:

1. **ADD** – Insert a new price level with the specified quantity
2. **CHANGE** – Modify the quantity at an existing price level  
3. **DELETE** – Remove an existing price level from the book
4. **DISCONNECT** – Clear the current OrderBook and mark the connection as inactive

### Disconnect Behavior

Any data blocks received while disconnected should not change the book. Instead, if a new snapshot is sent while disconnected, you must rebuild the book using that snapshot and apply any events whose DataBlock have a `seqNo` later than the snapshot's `asOfSeqNo`. It will be guaranteed that the `asOfSeqNo` is greater than or equal to the last DISCONNECT `seqNo`.

## Requirements

### DataBlock Processing
- DataBlocks must be processed sequentially (by `seqNo`) but are not guaranteed to be sent in order. This includes when the event stream is disconnected
- The first block to be processed will always have a sequence number of 1, and the next one should have one of 2
- ADD, CHANGE, and DELETE will affect bids if `isBuy` is True
- ADD, CHANGE, and DELETE will affect asks if `isBuy` is False
- CHANGE events can have a negative quantity
- DELETE events will have a value of `None` for their quantity
- DISCONNECT events will have a value of `None` for their price, quantity and isBuy

### Snapshot Processing
- Snapshots are always sent in order
- Snapshots can only be sent at the start or after a DISCONNECT
- A Snapshot is always sent at the start with an `asOfSeqNo` of 1
- You may want to add methods to them and you are free to do so

## Evaluation

Your implementation will be tested against the following test cases:

| Test | Description |
|------|-------------|
| `testAddEvent` | Single ADD DataBlock |
| `testChangeEvent` | Single CHANGE DataBlock |
| `testDeleteEvent` | Single DELETE DataBlock |
| `testAllEvents` | All possible DataBlock events besides DISCONNECT |
| `testDisconnect` | Multiple DataBlocks, a DISCONNECT event, multiple lost DataBlocks, and then a Snapshot |
| `testOutOfOrderEvents` | Multiple DataBlock events besides DISCONNECT received out of order |
| `testOutOfOrderLostMessages` | Multiple DataBlocks after a DISCONNECT received out of order |
| `testMultipleDisconnects` | Multiple DISCONNECT DataBlocks received out of order |

## Implementation Notes

- You can assume any DataBlock event is valid. For example, if we try to process a DELETE DataBlock the specified price will exist
- You should not add or modify any of the fields in DataBlock, OrderBook, and Snapshot. If you want to add methods to them and you are free to do so
- Upon a DISCONNECT event, only clear the existing OrderBook. DataBlocks received whose `seqNo` occur after the DISCONNECT should still be processed in sequence
- You should aim to outperform brute force search on an iterable whenever possible

## Your Task

Implement the `processMessage` method in the `EventStream` class to correctly handle all DataBlocks and Snapshots according to the requirements above.

In [2]:
# Write your solution here.
# Python version 3.12.3
## All the packages you need to complete this problem are already installed
## for you in our local (sandbox) virtual environment.

from dataclasses import dataclass
from enum import Enum, auto


type Quantity = int
type Price = float
type SeqNo = int


# DO NOT MODIFY
class Event(Enum):
    DISCONNECT = auto()
    ADD = auto()
    CHANGE = auto()
    DELETE = auto()

# DO NOT ADD OR DELETE ANY DATACLASS FIELDS
@dataclass
class DataBlock:
    event: Event
    price: Price 
    quantity: Quantity | None
    seqNo: SeqNo
    isBuy: bool | None


@dataclass
class OrderBook:
    bids: dict[Price | None, Quantity | None]
    asks: dict[Price | None, Quantity | None]

@dataclass
class Snapshot:
    book: OrderBook
    asOfSeqNo: SeqNo

class EventStream: 
    def __init__(self):
        self.orderBook: OrderBook = OrderBook({}, {})
        self.currentSeqNo: SeqNo = 1


    # Implement this
    def processMessage(self, msg: DataBlock | Snapshot) -> None:
        self.msgEvent(msg) if isinstance(msg, DataBlock) else None
        self.currentSeqNo = self.currentSeqNo + 1

    def msgEvent(self, dataBlock: DataBlock):
        event: Event = dataBlock.event
        match event:
            case Event.DISCONNECT:
              ss: Snapshot = Snapshot(self.orderBook, self.currentSeqNo)
              self.orderBook.asks = {}
              self.orderBook.bids = {}

            case Event.ADD:
              if dataBlock.isBuy:
                self.orderBook.bids[dataBlock.price] = dataBlock.quantity
              else:
                self.orderBook.asks[dataBlock.price] = dataBlock.quantity
              print(self.orderBook)

            case Event.CHANGE:
              price = dataBlock.price
              if dataBlock.isBuy and (price >= 0):
                  self.orderBook.bids[dataBlock.price] = dataBlock.quantity
              elif dataBlock.isBuy and (price < 0):
                  self.orderBook.bids[dataBlock.price] = (self.orderBook.bids[dataBlock.price] - abs(dataBlock.quantity)) # type: ignore
              elif not dataBlock.isBuy and (price >= 0): 
                  self.orderBook.asks[dataBlock.price] = dataBlock.quantity 
              elif not dataBlock.isBuy and (price < 0):
                  self.orderBook.asks[dataBlock.price] = (self.orderBook.asks[dataBlock.price] - abs(dataBlock.quantity)) # type: ignore
            case Event.DELETE:
              pass
            case _:
              pass

## Test Case 1: testAddEvent - Single ADD DataBlock

In [3]:
def testAddEvent():
    """Test single ADD DataBlock for both buy and sell sides"""
    event_stream = EventStream()
    
    # Initial snapshot with seqNo 1
    initial_snapshot = Snapshot(
        book=OrderBook(bids={}, asks={}),
        asOfSeqNo=1
    )
    event_stream.processMessage(initial_snapshot)
    
    # Test ADD for buy order (should create bid)
    add_buy = DataBlock(
        event=Event.ADD,
        price=100.0,
        quantity=50,
        seqNo=1,
        isBuy=True
    )
    event_stream.processMessage(add_buy)
    
    # Test ADD for sell order (should create ask)
    add_sell = DataBlock(
        event=Event.ADD,
        price=101.0,
        quantity=30,
        seqNo=2,
        isBuy=False
    )
    event_stream.processMessage(add_sell)
    
    # Verify the orderbook state
    expected_bids = {100.0: 50}
    expected_asks = {101.0: 30}
    
    assert event_stream.orderBook.bids == expected_bids, \
        f"Expected bids {expected_bids}, got {event_stream.orderBook.bids}"
    assert event_stream.orderBook.asks == expected_asks, \
        f"Expected asks {expected_asks}, got {event_stream.orderBook.asks}"
    
    print("✓ testAddEvent passed - Single ADD DataBlock correctly processed")
    return True

testAddEvent()

OrderBook(bids={100.0: 50}, asks={})
OrderBook(bids={100.0: 50}, asks={101.0: 30})
✓ testAddEvent passed - Single ADD DataBlock correctly processed


True

## Test Case 2: testChangeEvent - Single CHANGE DataBlock with negative quantity

In [4]:
def testChangeEvent():
    """Test single CHANGE DataBlock including negative quantities"""
    event_stream = EventStream()
    
    # Initial snapshot with existing orders
    initial_snapshot = Snapshot(
        book=OrderBook(
            bids={100.0: 50, 99.5: 40},
            asks={101.0: 30, 101.5: 20}
        ),
        asOfSeqNo=1
    )
    event_stream.processMessage(initial_snapshot)
    
    # Test CHANGE with positive quantity for buy order
    change_buy_positive = DataBlock(
        event=Event.CHANGE,
        price=100.0,
        quantity=75,  # Change from 50 to 75
        seqNo=1,
        isBuy=True
    )
    event_stream.processMessage(change_buy_positive)
    
    # Test CHANGE with negative quantity for sell order
    change_sell_negative = DataBlock(
        event=Event.CHANGE,
        price=101.0,
        quantity=-10,  # Negative quantity - should reduce from 30
        seqNo=2,
        isBuy=False
    )
    event_stream.processMessage(change_sell_negative)
    
    # Test CHANGE with negative quantity for buy order
    change_buy_negative = DataBlock(
        event=Event.CHANGE,
        price=99.5,
        quantity=-15,  # Reduce from 40
        seqNo=3,
        isBuy=True
    )
    event_stream.processMessage(change_buy_negative)
    
    # Verify the orderbook state
    expected_bids = {100.0: 75, 99.5: 25}  # 40 - 15 = 25
    expected_asks = {101.0: 20, 101.5: 20}  # 30 - 10 = 20
    
    assert event_stream.orderBook.bids == expected_bids, \
        f"Expected bids {expected_bids}, got {event_stream.orderBook.bids}"
    assert event_stream.orderBook.asks == expected_asks, \
        f"Expected asks {expected_asks}, got {event_stream.orderBook.asks}"
    
    print("✓ testChangeEvent passed - CHANGE DataBlock with negative quantities correctly processed")
    return True

testChangeEvent()

AssertionError: Expected bids {100.0: 75, 99.5: 25}, got {100.0: 75, 99.5: -15}

## Test Case 3: testDeleteEvent - Single DELETE DataBlock

In [ ]:
def testDeleteEvent():
    """Test single DELETE DataBlock with None quantity"""
    event_stream = EventStream()
    
    # Initial snapshot with existing orders
    initial_snapshot = Snapshot(
        book=OrderBook(
            bids={100.0: 50, 99.5: 40, 99.0: 30},
            asks={101.0: 30, 101.5: 20, 102.0: 10}
        ),
        asOfSeqNo=1
    )
    event_stream.processMessage(initial_snapshot)
    
    # Test DELETE for buy order (quantity should be None)
    delete_buy = DataBlock(
        event=Event.DELETE,
        price=99.5,
        quantity=None,  # DELETE events have None quantity
        seqNo=1,
        isBuy=True
    )
    event_stream.processMessage(delete_buy)
    
    # Test DELETE for sell order
    delete_sell = DataBlock(
        event=Event.DELETE,
        price=102.0,
        quantity=None,  # DELETE events have None quantity
        seqNo=2,
        isBuy=False
    )
    event_stream.processMessage(delete_sell)
    
    # Verify the orderbook state - deleted price levels should be removed
    expected_bids = {100.0: 50, 99.0: 30}  # 99.5 deleted
    expected_asks = {101.0: 30, 101.5: 20}  # 102.0 deleted
    
    assert event_stream.orderBook.bids == expected_bids, \
        f"Expected bids {expected_bids}, got {event_stream.orderBook.bids}"
    assert event_stream.orderBook.asks == expected_asks, \
        f"Expected asks {expected_asks}, got {event_stream.orderBook.asks}"
    
    print("✓ testDeleteEvent passed - DELETE DataBlock correctly processed")
    return True

testDeleteEvent()

## Test Case 4: testAllEvents - All event types except DISCONNECT

In [ ]:
def testAllEvents():
    """Test all possible DataBlock events besides DISCONNECT in sequence"""
    event_stream = EventStream()
    
    # Initial snapshot
    initial_snapshot = Snapshot(
        book=OrderBook(bids={}, asks={}),
        asOfSeqNo=1
    )
    event_stream.processMessage(initial_snapshot)
    
    # 1. ADD events to create initial orderbook
    events = [
        DataBlock(Event.ADD, 99.0, 100, 1, True),   # Add bid at 99.0
        DataBlock(Event.ADD, 98.5, 50, 2, True),    # Add bid at 98.5
        DataBlock(Event.ADD, 101.0, 80, 3, False),  # Add ask at 101.0
        DataBlock(Event.ADD, 101.5, 60, 4, False),  # Add ask at 101.5
    ]
    
    # 2. CHANGE events to modify quantities
    events.extend([
        DataBlock(Event.CHANGE, 99.0, 150, 5, True),     # Increase bid at 99.0
        DataBlock(Event.CHANGE, 101.0, -30, 6, False),   # Decrease ask at 101.0 (negative)
        DataBlock(Event.CHANGE, 98.5, -20, 7, True),     # Decrease bid at 98.5 (negative)
    ])
    
    # 3. More ADD events
    events.extend([
        DataBlock(Event.ADD, 98.0, 75, 8, True),     # Add new bid
        DataBlock(Event.ADD, 102.0, 90, 9, False),   # Add new ask
    ])
    
    # 4. DELETE events to remove price levels
    events.extend([
        DataBlock(Event.DELETE, 98.5, None, 10, True),   # Delete bid at 98.5
        DataBlock(Event.DELETE, 101.5, None, 11, False), # Delete ask at 101.5
    ])
    
    # Process all events in sequence
    for event in events:
        event_stream.processMessage(event)
    
    # Verify final orderbook state
    expected_bids = {
        99.0: 150,  # Changed from 100 to 150
        98.0: 75    # Added new, 98.5 was deleted
    }
    expected_asks = {
        101.0: 50,  # Changed from 80 to 50 (80 - 30)
        102.0: 90   # Added new, 101.5 was deleted
    }
    
    assert event_stream.orderBook.bids == expected_bids, \
        f"Expected bids {expected_bids}, got {event_stream.orderBook.bids}"
    assert event_stream.orderBook.asks == expected_asks, \
        f"Expected asks {expected_asks}, got {event_stream.orderBook.asks}"
    
    print("✓ testAllEvents passed - All events (ADD, CHANGE, DELETE) correctly processed")
    return True

testAllEvents()

## Test Case 5: testDisconnect - DISCONNECT event with lost messages and snapshot recovery

In [ ]:
def testDisconnect():
    """Test DISCONNECT event, lost messages during disconnect, and snapshot recovery"""
    event_stream = EventStream()
    
    # Initial snapshot
    initial_snapshot = Snapshot(
        book=OrderBook(
            bids={100.0: 50, 99.5: 30},
            asks={101.0: 40, 101.5: 20}
        ),
        asOfSeqNo=1
    )
    event_stream.processMessage(initial_snapshot)
    
    # Process some events before disconnect
    pre_disconnect_events = [
        DataBlock(Event.ADD, 99.0, 60, 1, True),      # Add new bid
        DataBlock(Event.CHANGE, 100.0, 70, 2, True),  # Change existing bid
        DataBlock(Event.ADD, 102.0, 35, 3, False),    # Add new ask
    ]
    
    for event in pre_disconnect_events:
        event_stream.processMessage(event)
    
    # DISCONNECT event (should clear orderbook and mark as inactive)
    disconnect_event = DataBlock(
        event=Event.DISCONNECT,
        price=None,     # DISCONNECT has None price
        quantity=None,  # DISCONNECT has None quantity  
        seqNo=4,
        isBuy=None      # DISCONNECT has None isBuy
    )
    event_stream.processMessage(disconnect_event)
    
    # Verify orderbook is cleared after disconnect
    assert event_stream.orderBook.bids == {}, \
        f"Bids should be empty after DISCONNECT, got {event_stream.orderBook.bids}"
    assert event_stream.orderBook.asks == {}, \
        f"Asks should be empty after DISCONNECT, got {event_stream.orderBook.asks}"
    
    # Lost messages during disconnect (should NOT be processed while disconnected)
    lost_events = [
        DataBlock(Event.ADD, 98.0, 100, 5, True),     # Lost - should not process
        DataBlock(Event.CHANGE, 101.0, 80, 6, False), # Lost - should not process
        DataBlock(Event.DELETE, 99.5, None, 7, True), # Lost - should not process
    ]
    
    for event in lost_events:
        event_stream.processMessage(event)
    
    # Orderbook should still be empty (disconnected, so events ignored)
    assert event_stream.orderBook.bids == {}, \
        f"Bids should remain empty during disconnect, got {event_stream.orderBook.bids}"
    assert event_stream.orderBook.asks == {}, \
        f"Asks should remain empty during disconnect, got {event_stream.orderBook.asks}"
    
    # New snapshot after reconnection (asOfSeqNo > disconnect seqNo)
    recovery_snapshot = Snapshot(
        book=OrderBook(
            bids={100.5: 85, 100.0: 65},
            asks={101.0: 55, 101.5: 45}
        ),
        asOfSeqNo=7  # After the lost messages
    )
    event_stream.processMessage(recovery_snapshot)
    
    # Events after snapshot (seqNo > asOfSeqNo, should be processed)
    post_recovery_events = [
        DataBlock(Event.ADD, 99.8, 40, 8, True),       # Should process
        DataBlock(Event.CHANGE, 101.0, 60, 9, False),  # Should process
        DataBlock(Event.DELETE, 100.0, None, 10, True), # Should process
    ]
    
    for event in post_recovery_events:
        event_stream.processMessage(event)
    
    # Verify final orderbook state
    expected_bids = {
        100.5: 85,  # From snapshot
        99.8: 40    # Added after recovery, 100.0 deleted
    }
    expected_asks = {
        101.0: 60,  # Changed from 55 to 60
        101.5: 45   # From snapshot
    }
    
    assert event_stream.orderBook.bids == expected_bids, \
        f"Expected bids {expected_bids}, got {event_stream.orderBook.bids}"
    assert event_stream.orderBook.asks == expected_asks, \
        f"Expected asks {expected_asks}, got {event_stream.orderBook.asks}"
    
    print("✓ testDisconnect passed - DISCONNECT and recovery correctly handled")
    return True

testDisconnect()

## Test Case 6: testOutOfOrderEvents - Out of order event processing

In [ ]:
def testOutOfOrderEvents():
    """Test that events are processed in sequential order by seqNo, not arrival order"""
    event_stream = EventStream()
    
    # Initial snapshot
    initial_snapshot = Snapshot(
        book=OrderBook(bids={}, asks={}),
        asOfSeqNo=1
    )
    event_stream.processMessage(initial_snapshot)
    
    # Create events that will arrive out of order
    # These should be buffered and processed in seqNo order
    out_of_order_events = [
        DataBlock(Event.ADD, 101.5, 45, 4, False),    # seqNo 4 - arrives first
        DataBlock(Event.ADD, 99.0, 100, 1, True),     # seqNo 1 - arrives second
        DataBlock(Event.CHANGE, 99.0, 120, 5, True),  # seqNo 5 - arrives third
        DataBlock(Event.ADD, 101.0, 80, 3, False),    # seqNo 3 - arrives fourth
        DataBlock(Event.ADD, 98.5, 50, 2, True),      # seqNo 2 - arrives fifth
        DataBlock(Event.DELETE, 98.5, None, 7, True), # seqNo 7 - arrives sixth
        DataBlock(Event.CHANGE, 101.0, -30, 6, False),# seqNo 6 - arrives seventh
    ]
    
    # Process all events (implementation should sort by seqNo)
    for event in out_of_order_events:
        event_stream.processMessage(event)
    
    # Expected processing order: seqNo 1, 2, 3, 4, 5, 6, 7
    # 1. ADD 99.0 bid with 100
    # 2. ADD 98.5 bid with 50
    # 3. ADD 101.0 ask with 80
    # 4. ADD 101.5 ask with 45
    # 5. CHANGE 99.0 bid to 120
    # 6. CHANGE 101.0 ask by -30 (80 - 30 = 50)
    # 7. DELETE 98.5 bid
    
    expected_bids = {99.0: 120}  # 98.5 was deleted
    expected_asks = {101.0: 50, 101.5: 45}
    
    assert event_stream.orderBook.bids == expected_bids, \
        f"Expected bids {expected_bids}, got {event_stream.orderBook.bids}"
    assert event_stream.orderBook.asks == expected_asks, \
        f"Expected asks {expected_asks}, got {event_stream.orderBook.asks}"
    
    print("✓ testOutOfOrderEvents passed - Events correctly processed in sequential order")
    return True

testOutOfOrderEvents()

## Test Case 7: testOutOfOrderLostMessages - Out of order messages after DISCONNECT

In [ ]:
def testOutOfOrderLostMessages():
    """Test out of order messages received after DISCONNECT and snapshot recovery"""
    event_stream = EventStream()
    
    # Initial snapshot
    initial_snapshot = Snapshot(
        book=OrderBook(
            bids={100.0: 50},
            asks={101.0: 40}
        ),
        asOfSeqNo=1
    )
    event_stream.processMessage(initial_snapshot)
    
    # Process events before disconnect
    pre_disconnect = [
        DataBlock(Event.ADD, 99.5, 30, 1, True),
        DataBlock(Event.ADD, 101.5, 25, 2, False),
    ]
    
    for event in pre_disconnect:
        event_stream.processMessage(event)
    
    # DISCONNECT event
    disconnect = DataBlock(
        event=Event.DISCONNECT,
        price=None,
        quantity=None,
        seqNo=3,
        isBuy=None
    )
    event_stream.processMessage(disconnect)
    
    # Out of order messages received during disconnect
    # These arrive out of order and should be buffered
    out_of_order_lost = [
        DataBlock(Event.ADD, 98.0, 70, 6, True),      # seqNo 6
        DataBlock(Event.CHANGE, 100.0, 90, 4, True),  # seqNo 4 
        DataBlock(Event.DELETE, 99.5, None, 8, True), # seqNo 8
        DataBlock(Event.ADD, 102.0, 55, 5, False),    # seqNo 5
        DataBlock(Event.CHANGE, 101.0, -15, 7, False),# seqNo 7
    ]
    
    for event in out_of_order_lost:
        event_stream.processMessage(event)
    
    # Orderbook should still be empty (disconnected)
    assert event_stream.orderBook.bids == {}, \
        f"Bids should be empty during disconnect, got {event_stream.orderBook.bids}"
    assert event_stream.orderBook.asks == {}, \
        f"Asks should be empty during disconnect, got {event_stream.orderBook.asks}"
    
    # Recovery snapshot with asOfSeqNo = 5
    # This means events 4 and 5 are included in snapshot
    # Events 6, 7, 8 should be applied after snapshot
    recovery_snapshot = Snapshot(
        book=OrderBook(
            bids={100.0: 90, 99.0: 45},  # Includes changes up to seqNo 5
            asks={101.0: 40, 102.0: 55}   # Includes seqNo 5 ADD
        ),
        asOfSeqNo=5
    )
    event_stream.processMessage(recovery_snapshot)
    
    # After snapshot, events with seqNo > 5 should be applied in order
    # That's events 6, 7, 8 from the buffered out of order messages
    # 6: ADD 98.0 bid with 70
    # 7: CHANGE 101.0 ask by -15 (40 - 15 = 25)
    # 8: DELETE 99.5 bid (not in snapshot, so no effect)
    
    expected_bids = {
        100.0: 90,  # From snapshot
        99.0: 45,   # From snapshot
        98.0: 70    # Applied after snapshot (seqNo 6)
    }
    expected_asks = {
        101.0: 25,  # Changed from 40 to 25 (seqNo 7)
        102.0: 55   # From snapshot
    }
    
    assert event_stream.orderBook.bids == expected_bids, \
        f"Expected bids {expected_bids}, got {event_stream.orderBook.bids}"
    assert event_stream.orderBook.asks == expected_asks, \
        f"Expected asks {expected_asks}, got {event_stream.orderBook.asks}"
    
    print("✓ testOutOfOrderLostMessages passed - Out of order messages after DISCONNECT correctly handled")
    return True

testOutOfOrderLostMessages()

## Test Case 8: testMultipleDisconnects - Multiple DISCONNECT events out of order

In [ ]:
def testMultipleDisconnects():
    """Test multiple DISCONNECT events received out of order"""
    event_stream = EventStream()
    
    # Initial snapshot
    initial_snapshot = Snapshot(
        book=OrderBook(
            bids={100.0: 50, 99.5: 40},
            asks={101.0: 30, 101.5: 20}
        ),
        asOfSeqNo=1
    )
    event_stream.processMessage(initial_snapshot)
    
    # Mix of events and disconnects arriving out of order
    mixed_events = [
        DataBlock(Event.ADD, 99.0, 60, 1, True),           # seqNo 1
        DataBlock(Event.DISCONNECT, None, None, 5, None),  # seqNo 5 - Second disconnect
        DataBlock(Event.ADD, 102.0, 35, 2, False),         # seqNo 2
        DataBlock(Event.DISCONNECT, None, None, 3, None),  # seqNo 3 - First disconnect
        DataBlock(Event.CHANGE, 100.0, 70, 4, True),       # seqNo 4 - Between disconnects
        DataBlock(Event.ADD, 98.5, 80, 6, True),           # seqNo 6 - After second disconnect
        DataBlock(Event.DISCONNECT, None, None, 8, None),  # seqNo 8 - Third disconnect
        DataBlock(Event.DELETE, 99.0, None, 7, True),      # seqNo 7 - Between 2nd and 3rd disconnect
    ]
    
    # Process all events (should be sorted by seqNo)
    for event in mixed_events:
        event_stream.processMessage(event)
    
    # Expected sequence:
    # 1. ADD 99.0 bid with 60
    # 2. ADD 102.0 ask with 35
    # 3. DISCONNECT (clear book, mark inactive)
    # 4. CHANGE 100.0 (ignored - disconnected)
    # 5. DISCONNECT (already disconnected, stays cleared)
    # 6. ADD 98.5 (ignored - still disconnected)
    # 7. DELETE 99.0 (ignored - still disconnected)
    # 8. DISCONNECT (already disconnected, stays cleared)
    
    # After all events, orderbook should be empty (last state is disconnected)
    assert event_stream.orderBook.bids == {}, \
        f"Bids should be empty after multiple disconnects, got {event_stream.orderBook.bids}"
    assert event_stream.orderBook.asks == {}, \
        f"Asks should be empty after multiple disconnects, got {event_stream.orderBook.asks}"
    
    # Recovery snapshot after all disconnects
    recovery_snapshot = Snapshot(
        book=OrderBook(
            bids={100.5: 90, 100.0: 70},
            asks={101.0: 50, 101.5: 40}
        ),
        asOfSeqNo=8  # After all events including disconnects
    )
    event_stream.processMessage(recovery_snapshot)
    
    # New events after recovery (seqNo > 8)
    post_recovery = [
        DataBlock(Event.ADD, 99.8, 55, 9, True),
        DataBlock(Event.CHANGE, 101.0, 65, 10, False),
        DataBlock(Event.DELETE, 100.0, None, 11, True),
    ]
    
    for event in post_recovery:
        event_stream.processMessage(event)
    
    # Verify final state
    expected_bids = {
        100.5: 90,  # From snapshot
        99.8: 55    # Added after recovery, 100.0 deleted
    }
    expected_asks = {
        101.0: 65,  # Changed from 50 to 65
        101.5: 40   # From snapshot
    }
    
    assert event_stream.orderBook.bids == expected_bids, \
        f"Expected bids {expected_bids}, got {event_stream.orderBook.bids}"
    assert event_stream.orderBook.asks == expected_asks, \
        f"Expected asks {expected_asks}, got {event_stream.orderBook.asks}"
    
    print("✓ testMultipleDisconnects passed - Multiple DISCONNECT events correctly handled")
    return True

testMultipleDisconnects()

## Test Suite Summary

All test cases have been created to thoroughly test the OrderBook Feed Handler implementation:

1. **testAddEvent**: Tests single ADD DataBlock operations for both buy and sell sides
2. **testChangeEvent**: Tests CHANGE DataBlock including negative quantities (requirement)
3. **testDeleteEvent**: Tests DELETE DataBlock with None quantity (requirement)
4. **testAllEvents**: Tests all event types (ADD, CHANGE, DELETE) in sequence
5. **testDisconnect**: Tests DISCONNECT clearing book, lost messages during disconnect, and snapshot recovery
6. **testOutOfOrderEvents**: Tests sequential processing by seqNo regardless of arrival order
7. **testOutOfOrderLostMessages**: Tests buffering and applying out-of-order messages after DISCONNECT
8. **testMultipleDisconnects**: Tests handling of multiple DISCONNECT events out of order

### Key Requirements Tested:
- ✅ DataBlocks processed sequentially by seqNo (not arrival order)
- ✅ First block has seqNo=1, incrementing by 1
- ✅ ADD/CHANGE/DELETE affect bids when isBuy=True, asks when isBuy=False
- ✅ CHANGE events can have negative quantities
- ✅ DELETE events have None quantity
- ✅ DISCONNECT has None for price, quantity, and isBuy
- ✅ DISCONNECT clears OrderBook and marks connection inactive
- ✅ Messages during disconnect are not processed
- ✅ Snapshot recovery rebuilds book and applies events with seqNo > asOfSeqNo
- ✅ Out-of-order event handling and buffering